# Feature engineering

## Data loading

In [2]:
import pandas as pd

path = './artifacts/saved_df.csv'
df = pd.read_csv(path, index_col=0)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3167 entries, 0 to 3166
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   type                 3167 non-null   object 
 1   date                 3167 non-null   object 
 2   time                 3167 non-null   object 
 3   duration             3167 non-null   object 
 4   distance             3167 non-null   object 
 5   origin               3167 non-null   object 
 6   destination          3167 non-null   object 
 7   total_earning        3167 non-null   float64
 8   base_fare            3167 non-null   float64
 9   customer_fare        3167 non-null   float64
 10  paid_to_driver       3167 non-null   float64
 11  paid_to_uber         3167 non-null   float64
 12  datetime             3167 non-null   object 
 13  duration_dt          3167 non-null   object 
 14  distance_km          3167 non-null   float64
 15  origin_lat_lng       3167 non-null   object

In [4]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['duration_dt'] = pd.to_timedelta(df['duration_dt'])

## Data exploration

In [5]:
df.head().T

,0,1,2,3,4
type,UberX,UberX,UberX,UberX,UberX
date,2023-11-11,2023-11-11,2023-11-11,2023-11-11,2023-11-11
time,17:28:00,17:20:00,16:37:00,16:19:00,15:57:00
duration,11 min 32 sec,0 sec,27 min 43 sec,16 min 18 sec,5 min 29 sec
distance,4.15 km,---,13.42 km,4.78 km,1.38 km
origin,"Rua Antônio das Chagas, Santo Amaro - São Paul...","Rua Luís Correia de Melo, Santo Amaro - São Pa...","Rua Helena, Itaim Bibi - São Paulo - SP, 04552...","Alameda Gabriel Monteiro da Silva, Jardim Amer...","R. Turiassu, Perdizes - São Paulo - SP, 05005-..."
destination,"R. Geórgia, Brooklin - São Paulo - SP, 04559-0...","Rua Luís Correia de Melo, Santo Amaro - São Pa...","Rua Quipa, Campo Limpo - São Paulo - SP, 05756...","Rua Helena, Itaim Bibi - São Paulo - SP, 04552...","Rua Ministro Godói, Perdizes - São Paulo - SP,..."
total_earning,15.01,4.22,27.09,13.78,6.69
base_fare,11.01,4.22,27.09,13.78,6.69
customer_fare,19.97,5.25,34.95,22.96,10.96


In [6]:
df['paid_to_driver'] = -1 * df['paid_to_driver']

In [7]:
df.describe()

,total_earning,base_fare,customer_fare,paid_to_driver,paid_to_uber,datetime,duration_dt,distance_km
count,3167.000000,3167.000000,3167.000000,3167.000000,3167.000000,3167,3167,3167.000000
mean,23.649678,22.317540,34.445595,24.281964,9.433423,2024-04-29 13:43:56.703505152,0 days 00:18:37.175244711,6.400751
min,-10.820000,1.890000,-5.070000,1.890000,-42.080000,2023-11-06 20:41:00,0 days 00:00:00,0.000000
25%,12.450000,11.625000,19.340000,12.945000,4.880000,2024-01-13 19:31:30,0 days 00:08:25.500000,2.270000
50%,18.920000,17.580000,28.270000,19.760000,7.980000,2024-04-16 15:31:00,0 days 00:14:25,4.140000
75%,29.410000,27.575000,43.140000,30.335000,12.640000,2024-08-13 19:12:00,0 days 00:24:09,7.955000
max,203.470000,203.470000,374.870000,209.470000,123.400000,2024-11-30 19:16:00,0 days 02:11:00,110.550000
std,16.639526,16.265912,23.131384,16.913488,8.456055,NaN,0 days 00:14:58.348358803,7.117789


In [8]:
df[df['total_earning'] < 0]

,type,date,time,duration,distance,origin,destination,total_earning,base_fare,customer_fare,paid_to_driver,paid_to_uber,datetime,duration_dt,distance_km,origin_lat_lng,destination_lat_lng
83,Comfort,2023-11-18,20:26:00,3 min 41 sec,0.74 km,"Avenida Paulista, São Paulo - São Paulo - SP, ...","Rua Haddock Lobo, Cerqueira César - São Paulo ...",-10.82,7.87,-5.07,9.18,5.75,2023-11-18 20:26:00,0 days 00:03:41,0.74,"[-23.5573402, -46.6612825]","[-23.5579545, -46.6617369]"


In [9]:
df[df['customer_fare'] < 0]

,type,date,time,duration,distance,origin,destination,total_earning,base_fare,customer_fare,paid_to_driver,paid_to_uber,datetime,duration_dt,distance_km,origin_lat_lng,destination_lat_lng
83,Comfort,2023-11-18,20:26:00,3 min 41 sec,0.74 km,"Avenida Paulista, São Paulo - São Paulo - SP, ...","Rua Haddock Lobo, Cerqueira César - São Paulo ...",-10.82,7.87,-5.07,9.18,5.75,2023-11-18 20:26:00,0 days 00:03:41,0.74,"[-23.5573402, -46.6612825]","[-23.5579545, -46.6617369]"


In [10]:
df.drop(df[df['total_earning'] < 0].index, inplace=True)

In [11]:
df[df['total_earning'] < 0]

,type,date,time,duration,distance,origin,destination,total_earning,base_fare,customer_fare,paid_to_driver,paid_to_uber,datetime,duration_dt,distance_km,origin_lat_lng,destination_lat_lng


In [12]:
df[df['paid_to_uber'] < 0]

,type,date,time,duration,distance,origin,destination,total_earning,base_fare,customer_fare,paid_to_driver,paid_to_uber,datetime,duration_dt,distance_km,origin_lat_lng,destination_lat_lng
7,UberX,2023-11-11,12:32:00,21 min 33 sec,5.28 km,"Rua Capital Federal, Perdizes - São Paulo - SP...","Av. Rebouças, Bairro Pinheiros - São Paulo - S...",20.00,20.00,24.88,20.00,-1.34,2023-11-11 12:32:00,0 days 00:21:33,5.28,"[-23.5407904, -46.6851003]","[-23.5645767, -46.678679]"
32,UberX,2023-11-09,19:33:00,9 min 51 sec,2.42 km,"Rua Roma, Lapa - São Paulo - SP, 05050-090, BR","Rua Bica de Pedra, Perdizes - São Paulo - SP, ...",12.02,11.02,13.37,12.02,-1.07,2023-11-09 19:33:00,0 days 00:09:51,2.42,"[-23.5227259, -46.7017216]","[-23.5398646, -46.6965347]"
51,UberX,2023-11-08,07:29:00,1 hr 5 min,15.86 km,"Avenida Doutor Salomão Vasconcelos, Cangaiba -...","Rua Florêncio de Abreu, Sé - São Paulo - SP, 0...",40.06,38.81,42.88,40.06,-5.13,2023-11-08 07:29:00,0 days 01:05:00,15.86,"[-23.504126, -46.524001]","[-23.539276, -46.6329022]"
74,UberX,2023-11-19,01:36:00,8 min 11 sec,4.22 km,"Rua Maria Amália Lopes Azevedo, Tremembé - São...","Rua Albertina V da Silva Gordo, Mandaqui - São...",16.00,13.50,17.59,16.00,-1.10,2023-11-19 01:36:00,0 days 00:08:11,4.22,"[-23.4564712, -46.5939624]","[-23.475604, -46.6236056]"
117,UberX,2023-11-15,21:19:00,21 min 31 sec,13.88 km,"Rua Calciolândia, Vila Medeiros - São Paulo - ...","R. Pedro de Castillo, Furnas - São Paulo - SP,...",30.01,30.01,32.21,30.01,-5.11,2023-11-15 21:19:00,0 days 00:21:31,13.88,"[-23.4935981, -46.5817175]","[-23.4198084, -46.5814597]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3076,Black,2024-11-21,09:38:00,24 min 12 sec,8.58 km,"Rua Califórnia, Itaim Bibi - São Paulo - SP, 0...","Avenida Brigadeiro Faria Lima, Pinheiros - São...",33.99,33.99,38.05,33.99,-1.88,2024-11-21 09:38:00,0 days 00:24:12,8.58,"[-23.6143072, -46.6866123]","[-23.5687776, -46.6920303]"
3096,Comfort,2024-11-18,19:25:00,55 min 39 sec,23.04 km,"Rua Pedroso Alvarenga, Itaim Bibi - São Paulo ...","Rua Domiciano Rossi, Centro - São Bernardo do ...",63.54,60.54,70.35,63.54,-0.21,2024-11-18 19:25:00,0 days 00:55:39,23.04,"[-23.5823143, -46.6803114]","[-23.6933196, -46.5624535]"
3108,VIP,2024-11-30,18:32:00,18 min 4 sec,6.41 km,"Avenida Doutor Altino Arantes, Saúde - Sao Pau...","Rua Xavier Curado, Ipiranga - Sao Paulo - SP, ...",23.53,20.53,27.53,23.53,-0.13,2024-11-30 18:32:00,0 days 00:18:04,6.41,"[-23.6026846, -46.6469652]","[-23.5844639, -46.604917]"
3116,Black,2024-11-29,18:57:00,13 min 25 sec,2.81 km,"Alameda Tietê, Jardim Paulista - Sao Paulo - S...","Rua Fradique Coutinho, Pinheiros - Sao Paulo -...",22.10,17.21,25.86,22.10,-1.41,2024-11-29 18:57:00,0 days 00:13:25,2.81,"[-23.5611184, -46.6667842]","[-23.5645961, -46.6854736]"


It seems that total_earning does not correspond to how much the driver earned. It's better to use paid_to_driver as total earning.

In [13]:
# Split df in paid rides and canceled ones.
df_paid_cancelations = df[df['distance_km']<= 0]
df_rides = df.drop(df_paid_cancelations.index)

## Feature engineering

### Formation of base benchmarks:
- fee = paid_to_uber / customer_fare
- earning_pct = paid_to_driver / customer_fare
- earning_base_fare_pct = base_fare / customer_fare
- traffic_benchmark = duration / distance
- ted = paid_to_driver / duration
- bfd = base_fare / duration
- cfd = customer_fare / duration 
- pud = paid_to_uber / duration 
- tei = paid_to_driver / distance
- bfi = base_fare / distance
- cfi = customer_fare / distance 
- pui = paid_to_uber / distance
- tedi = paid_to_driver / (duration * distance)
- bfdi = base_fare / (duration * distance)
- cfdi = customer_fare / (duration * distance) 
- pudi = paid_to_uber / (duration * distance) 

In [14]:
df_rides['fee'] = df_rides['paid_to_uber'] / df_rides['customer_fare'] * 100
df_rides['earning_pct'] = df_rides['paid_to_driver'] / df_rides['customer_fare'] * 100
df_rides['earning_base_fare_pct'] = df_rides['base_fare'] / df_rides['customer_fare'] * 100
df_rides['traffic_benchmark'] = (df_rides['duration_dt'].dt.total_seconds()/60) / df_rides['distance_km']
df_rides['ted'] = df_rides['paid_to_driver'] / (df_rides['duration_dt'].dt.total_seconds()/60)
df_rides['bfd'] = df_rides['base_fare'] / (df_rides['duration_dt'].dt.total_seconds()/60)
df_rides['cfd'] = df_rides['customer_fare'] / (df_rides['duration_dt'].dt.total_seconds()/60)
df_rides['pud'] = df_rides['paid_to_uber'] / (df_rides['duration_dt'].dt.total_seconds()/60)
df_rides['tei'] = df_rides['paid_to_driver'] / df_rides['distance_km']
df_rides['bfi'] = df_rides['base_fare'] / df_rides['distance_km']
df_rides['cfi'] = df_rides['customer_fare'] / df_rides['distance_km']
df_rides['pui'] = df_rides['paid_to_uber'] / df_rides['distance_km']
df_rides['tedi'] = df_rides['paid_to_driver'] / ((df_rides['duration_dt'].dt.total_seconds()/60) * df_rides['distance_km'])
df_rides['bfdi'] = df_rides['base_fare'] / ((df_rides['duration_dt'].dt.total_seconds()/60) * df_rides['distance_km'])
df_rides['cfdi'] = df_rides['customer_fare'] / ((df_rides['duration_dt'].dt.total_seconds()/60) * df_rides['distance_km'])
df_rides['pudi'] = df_rides['paid_to_uber'] / ((df_rides['duration_dt'].dt.total_seconds()/60) * df_rides['distance_km'])

In [15]:
df_rides.T

,0,2,3,4,5,6,7,8,9,10,...,3157,3158,3159,3160,3161,3162,3163,3164,3165,3166
type,UberX,UberX,UberX,UberX,UberX,UberX,UberX,UberX,UberX,UberX,...,UberX,UberX,Comfort,Comfort,Comfort,Black,Black,Black,Black,Black
date,2023-11-11,2023-11-11,2023-11-11,2023-11-11,2023-11-11,2023-11-11,2023-11-11,2023-11-11,2023-11-11,2023-11-11,...,2024-11-25,2024-11-25,2024-11-25,2024-11-25,2024-11-25,2024-11-25,2024-11-25,2024-11-25,2024-11-25,2024-11-25
time,17:28:00,16:37:00,16:19:00,15:57:00,15:01:00,13:06:00,12:32:00,12:28:00,12:18:00,12:10:00,...,18:27:00,17:05:00,16:00:00,14:43:00,14:11:00,08:33:00,08:17:00,08:03:00,07:48:00,06:49:00
duration,11 min 32 sec,27 min 43 sec,16 min 18 sec,5 min 29 sec,2 min 54 sec,4 min 5 sec,21 min 33 sec,3 min 59 sec,1 min 54 sec,4 min 0 sec,...,27 min 0 sec,1 hr 20 min,51 min 54 sec,1 hr 8 min,25 min 27 sec,4 min 48 sec,7 min 49 sec,7 min 36 sec,7 min 42 sec,15 min 11 sec
distance,4.15 km,13.42 km,4.78 km,1.38 km,0.96 km,0.80 km,5.28 km,0.91 km,0.55 km,1.61 km,...,8.12 km,25.80 km,28.58 km,29.37 km,8.19 km,1.23 km,1.10 km,1.64 km,1.35 km,4.76 km
origin,"Rua Antônio das Chagas, Santo Amaro - São Paul...","Rua Helena, Itaim Bibi - São Paulo - SP, 04552...","Alameda Gabriel Monteiro da Silva, Jardim Amer...","R. Turiassu, Perdizes - São Paulo - SP, 05005-...","Rua das Tabocas, Alto de Pinheiros - São Paulo...","Terminal Butantã, São Paulo - SP, 05510-050, BR","Rua Capital Federal, Perdizes - São Paulo - SP...","R. Francisco Isoldi, Alto de Pinheiros - São P...","Rua Judite, Alto de Pinheiros - São Paulo - SP...","Av. Arruda Botelho, Alto de Pinheiros - São Pa...",...,"Rua Dona Ana Néri, Cambuci - São Paulo - SP, 0...","Rua dos Têxteis, Cidade Tiradentes - São Paulo...","Terminal 2, Aeroporto Internacional de São Pau...","Rua Martim Francisco, Consolação - Sao Paulo -...","Rua Itaici, Santana - São Paulo - SP, 02460-03...","R. Fradique Coutinho, Pinheiros - São Paulo - ...","R. Fradique Coutinho, São Paulo - SP, 05422-00...","Rua Francisco Leitão, Pinheiros - Sao Paulo - ...","Rua Lisboa, Jardim Paulista - São Paulo - SP, ...","Rua Alves Guimarães, Jardim Paulista - São Pau..."
destination,"R. Geórgia, Brooklin - São Paulo - SP, 04559-0...","Rua Quipa, Campo Limpo - São Paulo - SP, 05756...","Rua Helena, Itaim Bibi - São Paulo - SP, 04552...","Rua Ministro Godói, Perdizes - São Paulo - SP,...","R. Delfina, Vila Madalena - São Paulo - SP, 05...","Avenida Afrânio Peixoto, Butantã - São Paulo -...","Av. Rebouças, Bairro Pinheiros - São Paulo - S...","Rua Paulistânia, Sumarezinho - São Paulo - SP,...","Rua Gumercindo Fleury, Alto de Pinheiros - São...","Pç. Panamericana, Alto de Pinheiros - São Paul...",...,"Rua Doutor Olavo Egídio, Santana - São Paulo -...","Rua Vitoantônio Del Vechio, Mooca - São Paulo ...","Rua Ernesto Gould, Cidade Tiradentes - São Pau...","Terminal 3, Aeroporto Internacional de São Pau...","Rua Marquês de Itu, Consolação - São Paulo - S...","Rua Manuel Henrique Lopes, Pinheiros - São Pau...","R. Mourato Coelho Sobreloja, Vila Madalena - S...","R. Simão Álvares, Pinheiros - São Paulo - SP, ...","Av. Brasil, Jardins - São Paulo - SP, 01431-01...","R. Dona Veridiana, Higienópolis - São Paulo - ..."
total_earning,15.01,27.09,13.78,6.69,6.11,9.52,20.0,6.7,6.11,6.89,...,27.52,54.96,76.38,76.52,26.02,15.46,13.76,15.53,16.82,24.94
base_fare,11.01,27.09,13.78,6.69,6.11,6.52,20.0,6.7,6.11,6.89,...,25.02,54.96,74.35,75.09,26.02,9.96,9.51,11.03,10.32,20.3
customer_fare,19.97,34.95,22.96,10.96,9.98,10.9,24.88,10.97,9.94,10.99,...,30.3,60.23,90.31,128.42,43.37,18.49,16.02,17.89,22.13,33.75


In [16]:
# Some statistics of engineered features.
df_rides.iloc[:,-16:].describe().T

,count,mean,std,min,25%,50%,75%,max
fee,3106.0,27.890808,14.827361,-85.273091,21.227895,34.550489,38.794046,41.131956
earning_pct,3106.0,70.028322,10.842473,32.239156,61.055754,65.135683,77.064336,133.773165
earning_base_fare_pct,3106.0,63.643565,9.462541,23.446659,59.974899,61.765504,66.603208,133.773165
traffic_benchmark,3106.0,3.587073,1.654064,0.805066,2.602176,3.348431,4.287037,50.833333
ted,3106.0,1.536228,0.813015,0.433043,1.107823,1.378844,1.723635,22.500000
bfd,3106.0,1.371537,0.584133,0.391726,1.035075,1.270874,1.559897,11.400000
cfd,3106.0,2.211992,1.073666,0.636702,1.609723,2.020268,2.531542,24.648000
pud,3106.0,0.625579,0.496719,-5.820000,0.366796,0.628633,0.860244,5.409836
tei,3106.0,5.553911,10.546732,1.161911,3.498484,4.641873,6.172056,476.000000
bfi,3106.0,4.910562,9.218860,1.161911,3.291346,4.324893,5.495495,476.000000


With these features we can have a better understanding of the data and the relationships between the variables. Now, let's generate some visualizations to understand the data better.

In [17]:
df_rides.to_csv('./artifacts/df_rides.csv')
df_paid_cancelations.to_csv('./artifacts/df_paid_cancelations.csv')